# Import libraries

In [2]:
import numpy as np
import cv2
import os

In [3]:
lbpcascade_frontalface = 'opencv_xml_files/lbpcascade_frontalface.xml'

# Detect face using Lbph Cascade

In [4]:
def detect_face(input_img):
    """
    detect faces from an input image
    return: detected face and its postions that is x,y,w,h
    """
    image = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(lbpcascade_frontalface)
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=4)
    if (len(faces) == 0):
        return -1, -1
    (x, y, w, h) = faces[0]
    return image[y:y+w, x:x+h], faces[0]

# Draw rectangular box and put label on detected face

In [5]:
def draw_rectangle(img, rect):
    """
    draws rectangular bounding box around detected face
    """
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

def draw_text(img, text, x, y):
    """
    put label above the box
    """
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

# open webcam and read each frame. Each frame is passed through face detection function and finally detected face is saved in a folder which will be used for training

In [6]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    _, frame = cap.read()
    face, rect = detect_face(frame)
    if face is not -1:
        count +=1
        draw_rectangle(frame, rect)
        # change the folder path as per requirement
        cv2.imwrite("detected_faces/1/"+str(count)+ ".jpg", face)
    cv2.imshow("frame", frame)
    # destroy window on pressing key 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 

# Prepare training data: Read images from folder and prepare training set and respective labels

In [7]:
detected_faces = []
face_labels = []

def prepare_training_data(training_data_folder_path):
    """
    read images from folder and prepare training dataset
    return list of detected face and labels
    """
    traning_image_dirs = os.listdir(training_data_folder_path)
    for dir_name in traning_image_dirs:
        label = int(dir_name)
        training_image_path = training_data_folder_path + "/" + dir_name
        training_images_names = os.listdir(training_image_path)
        for image_name in training_images_names:
            image_path = training_image_path  + "/" + image_name
            image = cv2.imread(image_path, 0)
            detected_faces.append(image)
            face_labels.append(label)

    return detected_faces, face_labels

In [8]:
detected_faces, face_labels = prepare_training_data('detected_faces')

In [10]:
print("Total faces: ", len(detected_faces))
print("Total labels: ", len(face_labels))

Total faces:  124
Total labels:  124


# Initialize a face recongizer model from OpenCV module

In [11]:
lbphfaces_recognizer = cv2.face. LBPHFaceRecognizer_create(radius=1, neighbors=8)

# train the face recognizer model

In [12]:
lbphfaces_recognizer.train(detected_faces, np.array(face_labels))

# open webcam, read each frame and pass it to the trained model. For binary prediction: If the confidence value of prediction is less than the threshold value then set label text equal to user name otherwise unknown. For multiclass prediction: set label_text to label

In [14]:
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    face, rect = detect_face(frame)
    if face is not -1:
        label= lbphfaces_recognizer.predict(face)
        if label[1]<75:
            label_text = str(label[0])
        else:
            label_text = 'unknown'
        draw_rectangle(frame, rect)
        if label[1]<75:
            draw_text(frame, label_text, rect[0], rect[1]-5)
        else:
            draw_text(frame, label_text, rect[0], rect[1]-5)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()